In [1]:
import json
import os
import numpy as np
from tqdm import tqdm
from trankit import Pipeline

In [21]:
FILE = 'dataset/final_filtered_v0515.json'
OUT_FILE = 'dataset/final_filtered_sample_v0515_trankit.json'
DIR = 'dataset/sampled/'
filter_ =  ['....', 'Advertisement', 'ADVERTISEMENT', 'All rights reserved.']
media = ["Washington Times", "CNN (Online News)", "New York Times (News)", 'New York Times (News)',"Washington Post", 'BBC News']

In [40]:
import itertools

def generate_pairs(d, p):    
    
    d_dict = {}
    articles = d['articles']
    for a in d['articles']:
        art_content2 = []
        a_id = 'article_' + a['article_id']
        art = p.ssplit(a['content'])
        sents = [s['text'] for s in art['sentences']]
        sents.append(a['title'])
        art_content2.append(sents)
        d_dict[a_id] = sents

    key = ['premise', 'hypothesis']
    id_ = d['story_id']
    file_name = DIR + 'topic_' + str(id_) + '.json'
    whole_pairs = []
    
    cont_keys = d_dict.keys()
    key_pairs = list(itertools.combinations(cont_keys, 2))
    for pair in key_pairs:
        idx = pair[0] + '_' + pair[1]
        cont0 = d_dict[pair[0]]
        cont1 = d_dict[pair[1]]
        cont_pairs = list(itertools.product(cont0, cont1))
        for j, p in enumerate(cont_pairs):
            p_dict ={}
            idx_p1 = idx + '_'+ str(2*j)
            p_dict[key[0]] = p[0]
            p_dict[key[1]] = p[1]
            p_dict['idx'] = idx_p1
            whole_pairs.append(p_dict)
            p_dict = {}
            idx_p2 = idx + '_'+ str(2*j + 1)
            p_dict[key[0]] = p[1]
            p_dict[key[1]] = p[0]
            p_dict['idx'] = idx_p2
            whole_pairs.append(p_dict)
                
    with open(file_name, 'w', encoding='utf-8') as f:
        json.dump({"data":whole_pairs}, f)


In [52]:
def filter_data(data):

    p = Pipeline('english', embedding='xlm-roberta-large', gpu=True, cache_dir='/mnt/.cache')
    overall = []

    error = ['error code: 404', 'page not found']

    for d in tqdm(data):
    
        articles = []
    
        for a in d['articles']:
            
            Error = False
            
            if a['source'] in media:
                continue
                
            doc = p.ssplit(a['content'])
            sents = [s['text'] for s in doc['sentences']]
            sents = [s for s in sents if s not in filter_]
            print(len(sents))
            print(sents)
            
            if not 4 < len(sents) < 50:
                continue
            
            for e in error:
                if e in a['content'].lower():
                    Error = True
                    print(a['content'])
            if Error:
                continue
            articles.append(a)
            
            
        d['articles'] = articles
        if len(d['articles']) > 1:
            overall.append(d)
            generate_pairs(d, p)

    print(len(overall))
    return overall

In [ ]:
data = json.load(open(FILE, 'r')) 
new_data = filter_data(data)


In [43]:
with open(OUT_FILE, 'w') as f:
    json.dump(new_data, f)